Actions each agent have to decide between: Leave or Stay 

In [102]:
import numpy as np

class Agent: 
    def __init__(self, music_taste = 1, strategy = 1):
        self.music_taste = music_taste
        self.position_utility = 0
        self.accumulated_position_utility = 0
        self.waiting_utility = 0
        self.strategy = strategy
        self.position_history = []

    
class Concert:
    def __init__(self, len_rows = 1, music_type = 1, capacity = 5, duration = 2, nr_of_agents = 10):
        self.len_rows = len_rows
        self.music_type = music_type
        self.nr_agents = 0
        self.nr_waiting_agents = 0
        self.capacity = capacity
        self.nr_rows = np.ceil(capacity/len_rows)
        self.duration = duration #timesteps
        self.agents = np.zeros(nr_of_agents)

    def add_agent(self):
        self.nr_agents += 1

def get_placement_utility(concert):
    placement_factor = 5
    #continuing using sigmoid
    #fullness = concert.nr_agents / concert.capacity
    #sigmoid_x = 1 - fullness
    #k = 10 #steepness of sigomid
    #center = 0.5 #[0,1] with 0.1 happy crowd and 0.9 happy front row
    #utility = placement_factor * 1 / ( 1 + np.exp(-k*(sigmoid_x- center)))

    #stepwise by which row the agent is in, sensitive to concert row length
    dist_from_stage = np.floor((concert.nr_agents) / concert.len_rows)
    utility = placement_factor * (concert.nr_rows - dist_from_stage) / concert.nr_rows
    return utility

def get_time_at_concert_utility(agent):
    utility += agent

def get_waiting_utility(time):
    waiting_factor = 1
    utility = -time * waiting_factor
    return utility

Strategies: Sart with defining strategies that do not depend on any history. Deterministic depending on which timestep it likes to leave

In [103]:
def get_action(agent, timestep, max_time):
    # Stay at the consert
    if (agent.strategy == 1):
        if (timestep < max_time-1):
            action = 'stay'
        else:
            action = 'leave'
        
    # Leave to the next consert as early as possible
    if (agent.strategy == 2):
        if (timestep < max_time-1):
            action = 'leave'
        else:
            action = 'stay'

    # The one that likes to skip concerts 
    if (agent.strategy == 3):
        if (timestep < 1):
            action = 'stay'
        else:
            action = 'skip'

    return action


Inintialize agents and concerts 

In [104]:
nr_of_agents = 100
agents = []

nr_conserts = 1000
concerts = []

for _ in range (nr_of_agents):
    agents.append(Agent(music_taste=np.random.randint(1,4), strategy=np.random.randint(1,4)))

for _ in range(nr_conserts):
    concerts.append(Concert(music_type=np.random.randint(1,3), capacity = nr_of_agents, duration = 5, nr_of_agents = nr_of_agents))

#Add all agents to the starting concerts in a random order
shuffled_indices = np.random.permutation(nr_of_agents)
for i in range (nr_of_agents):
    idx = shuffled_indices[i]
    position_utility = 0 #get_placement_utility(concerts[0])
    concerts[0].add_agent() 
    concerts[0].agents[idx] = 1
    agents[idx].position_utility = position_utility
    agents[idx].accumulated_position_utility = position_utility
    concerts[0].add_agent() 


In [105]:
for i in range(0,len(concerts)-2):  # assume everyone stays at the last concert
    current_concert = concerts[i]
    next_concert = concerts[i + 1]
    next_next_concert = concerts[ i + 2]

    counter = 0

    for timestep in range(0,current_concert.duration):

        shuffled_indices = np.random.permutation(nr_of_agents)
        for j in range (nr_of_agents):
            idx = shuffled_indices[j]

            if (current_concert.agents[idx] == 0):
                continue
            
            action = get_action(agents[idx],timestep, current_concert.duration)
            
            if (action == 'leave'):
                #TODO All agents leaving during the same timestep must get the same utility
                agents[idx].position_history.append(agents[idx].accumulated_position_utility) #rätt tillfälle?
                agents[idx].waiting_utility += get_waiting_utility(current_concert.duration - timestep) 
                position_utility = get_placement_utility(next_concert)
                next_concert.add_agent() 
                next_concert.agents[idx] = 1
                agents[idx].position_utility = position_utility
                agents[idx].accumulated_position_utility = position_utility
                current_concert.agents[idx] = 0

            elif (action == 'skip'):
                agents[idx].position_history.append(agents[idx].accumulated_position_utility) #rätt tillfälle?
                agents[idx].waiting_utility += get_waiting_utility(next_next_concert.duration)
                position_utility = get_placement_utility(next_next_concert)
                next_next_concert.add_agent() #TODO: ability to skip a concert
                next_next_concert.agents[idx] = 1
                agents[idx].position_utility = position_utility
                agents[idx].accumulated_position_utility = position_utility
                current_concert.agents[idx] = 0
                
            else: #they stay
                agents[idx].accumulated_position_utility += agents[idx].position_utility
                



How did the simulation go?

In [106]:
sorted_agents = sorted(agents, key=lambda agent: sum(agent.position_history) + agent.waiting_utility, reverse=True)
for i in range (nr_of_agents):
    print("Utility: ", sum(sorted_agents[i].position_history) + sorted_agents[i].waiting_utility, ", Strategy: ", sorted_agents[i].strategy)

Utility:  6994.25 , Strategy:  1
Utility:  6961.5 , Strategy:  1
Utility:  6960.75 , Strategy:  1
Utility:  6954.25 , Strategy:  1
Utility:  6938.5 , Strategy:  1
Utility:  6934.5 , Strategy:  1
Utility:  6919.25 , Strategy:  1
Utility:  6917.75 , Strategy:  1
Utility:  6916.25 , Strategy:  1
Utility:  6916.25 , Strategy:  1
Utility:  6898.25 , Strategy:  1
Utility:  6877.75 , Strategy:  1
Utility:  6877.5 , Strategy:  1
Utility:  6869.75 , Strategy:  1
Utility:  6856.75 , Strategy:  1
Utility:  6854.75 , Strategy:  1
Utility:  6844.75 , Strategy:  1
Utility:  6842.0 , Strategy:  1
Utility:  6838.5 , Strategy:  1
Utility:  6833.5 , Strategy:  1
Utility:  6827.75 , Strategy:  1
Utility:  6818.25 , Strategy:  1
Utility:  6815.75 , Strategy:  1
Utility:  6812.0 , Strategy:  1
Utility:  6807.75 , Strategy:  1
Utility:  6802.5 , Strategy:  1
Utility:  6801.25 , Strategy:  1
Utility:  6797.75 , Strategy:  1
Utility:  6797.0 , Strategy:  1
Utility:  6739.5 , Strategy:  1
Utility:  6701.75 , S

In [107]:
agents_1 = []
agents_2 = []
agents_3 = []
for agent in agents:
    if agent.strategy == 1 : 
        agents_1.append(agent)
    elif agent.strategy == 2:
        agents_2.append(agent)
    else:
        agents_3.append(agent)

agents_1_utility = [0]
agents_2_utility = [0]
agents_3_utility = [0]

for i in range(0,nr_conserts-1):
    #undone
    sum(agents_1[:].position_history[i])

AttributeError: 'list' object has no attribute 'position_history'